In [5]:
import plotly.express as px
import pandas as pd

In [6]:
df = pd.read_csv('conferencemap.csv')

print(f"Map created with {len(df)} academic events from {df['Year'].min()} to {df['Year'].max()}")
print(f"Countries represented: {', '.join(sorted(df['Country'].unique()))}")
df

Map created with 21 academic events from 2017 to 2025
Countries represented:  UK, France, Germany, Ireland, Lithuania, Spain, UK


City    Country  Latitude  Longitude  \
0   Banyuls-sur-Mer     France   42.4817     3.1317   
1           Vilnius  Lithuania   54.6872    25.2797   
2             Mainz    Germany   49.9929     8.2473   
3         Lancaster         UK   54.0098    -2.7875   
4          Grenoble     France   45.1885     5.7245   
5         Lancaster         UK   54.0098    -2.7875   
6          Limerick    Ireland   52.6638    -8.6267   
7         Cambridge         UK   52.2055     0.1210   
8            London         UK   51.5074    -0.1278   
9         Lancaster         UK   54.0098    -2.7875   
10             Vigo      Spain   42.2407    -8.7207   
11        Lancaster         UK   54.0098    -2.7875   
12       Manchester         UK   53.4630    -2.2320   
13       Manchester         UK   53.4709    -2.2953   
14        Lancaster         UK   54.0098    -2.7875   
15         Augsburg    Germany   48.3529    10.8865   
16        Edge Hill         UK   53.5583    -2.8692   
17          Glasgow         UK   55.8718    -4.2882   
18       Birmingham         UK   52.4866    -1.8901   
19         Bayreuth    Germany   49.9478    11.5789   
20       Hildesheim    Germany   52.1507     9.9511   

                                                Event  Year        Date  \
0              École thématique d'été « Annotations »  2022  2022-06-01   
1                                            ICAME 46  2025  2025-06-01   
2                  Brexit means Brexit? Ein Symposium  2017  2017-12-01   
3                BAAL Corpus Linguistics SIG Workshop  2019  2019-11-01   
4          Advanced Language Processing Winter School  2021  2021-01-01   
5                      Machine Learning for Humanists  2021  2021-06-01   
6                                             CL 2021  2021  2021-07-01   
7                                            ICAME 43  2022  2022-07-01   
8                                            CHIMED-3  2023  2023-05-01   
9                                              CL2023  2023  2023-07-01   
10                                           ICAME 45  2024  2024-06-01   
11                       UCREL NLP Summer School 2024  2024  2024-07-01   
12           Towards a National Collection Conference  2024  2024-11-01   
13                                  BEYOND Conference  2024  2024-11-01   
14                           Digital Humanities Forum  2024  2024-12-01   
15                               HLT Augsburg Retreat  2025  2025-04-01   
16                    Edge Hill Corpus Research Group  2023  2023-12-01   
17  Digital Preservation Toolkit for Community Arc...  2025     2025-02   
18                                             CL2025  2025     2025-07   
19                                                CMC  2025  2025-09-03   
20                                            KONVENS  2025  2025-09-09   

                                           Full_Title  
0              École thématique d'été « Annotations »  
1                                            ICAME 46  
2   Brexit means Brexit? Ein Symposium - Akademie ...  
3   BAAL Corpus Linguistics Special Interest Group...  
4   Advanced Language Processing Winter School - ALPS  
5             Machine Learning for Humanists - N8 CIR  
6   The International Corpus Linguistics Conferenc...  
7   The 43rd Annual Conference of the Internationa...  
8   CHIMED-3, 3rd International Conference on Hist...  
9   CL2023, 12th International Corpus Linguistics ...  
10  The 45th Annual Conference of the Internationa...  
11                       UCREL NLP Summer School 2024  
12           Towards a National Collection Conference  
13                                  BEYOND Conference  
14                           Digital Humanities Forum  
15                               HLT Augsburg Retreat  
16                    Edge Hill Corpus Research Group  
17  Digital Preservation Toolkit for Community Arc...  
18  CL2025, 13th International Corpus Linguistics ...  
19  The 12th International Confer

In [7]:
# Create a new column for location
df['Location'] = df['City'] + ', ' + df['Country']
# Create a new column for event count
df['Event Count'] = df.groupby('Location')['Event'].transform('count')

# Group events by location and concatenate event titles
df_grouped = df.groupby(['Location', 'Latitude', 'Longitude'])['Event'].apply(lambda x: '<br> '.join(x)).reset_index()
df_years = df.groupby(['Location', 'Latitude', 'Longitude'])['Year'].apply(lambda x: ', '.join(map(str, x))).reset_index()

# Merge the grouped data with the original data
df = df.merge(df_grouped, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_grouped'))
df = df.merge(df_years, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_years'))
df['Is_Past'] = df['Date'].apply(lambda x: pd.to_datetime(x) < pd.Timestamp('today'))

In [8]:


# Create the map
fig = px.scatter_map(
    df,
    lat='Latitude',
    lon='Longitude',
    hover_name='Location',
    hover_data={
        'Location': True,
        'Event_grouped': True,
        'Year_years': True
    },
    size='Event Count',
    width=800,
    height=600,
    zoom=3.2,
    map_style= 'carto-positron'
    )

# Update marker styling
fig.update_traces(
    marker=dict(
        opacity=0.55,
        color=[ "#7b61a7" if x else "orange" for x in df["Is_Past"]]
    ),
    hovertemplate='<b>%{customdata[0]}</b>'+
                  '<br>%{customdata[1]}'+
                  '<br>%{customdata[2]}'
)

# Update layout for better aesthetics
fig.update_layout(
    font=dict(size=12),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False  # Hide the legend
)

# Show the map
fig.show()
fig.write_html("conferencemap.html")
